In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Bitcoin Node Proxy\n",
    "\n",
    "This notebook demonstrates how to use the `NodeProxy` class from the `bitcoinutils` library to interact with a Bitcoin node."
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Setup\n",
    "\n",
    "First, let's import the necessary modules and set up the network."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "from bitcoinutils.setup import setup\n",
    "from bitcoinutils.proxy import NodeProxy\n",
    "\n",
    "# Always remember to setup the network\n",
    "setup('testnet')"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Connecting to a Bitcoin Node\n",
    "\n",
    "To connect to a Bitcoin node, you need to create a `NodeProxy` instance with the appropriate credentials."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Replace these with your actual RPC credentials\n",
    "RPC_USER = 'your_rpc_username'\n",
    "RPC_PASSWORD = 'your_rpc_password'\n",
    "\n",
    "# Create a NodeProxy instance\n",
    "# If you're running Bitcoin Core locally, you can use the default host and port\n",
    "proxy = NodeProxy(RPC_USER, RPC_PASSWORD)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Getting Blockchain Information\n",
    "\n",
    "Let's get some basic information about the blockchain."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Get the current block count\n",
    "block_count = proxy.get_block_count()\n",
    "print(f\"Current block count: {block_count}\")\n",
    "\n",
    "# Get detailed blockchain information\n",
    "blockchain_info = proxy.get_blockchain_info()\n",
    "print(f\"Chain: {blockchain_info['chain']}\")\n",
    "print(f\"Difficulty: {blockchain_info['difficulty']}\")\n",
    "print(f\"Median time: {blockchain_info['mediantime']}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Working with Blocks\n",
    "\n",
    "Now let's retrieve information about a specific block."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Get the hash of the latest block\n",
    "latest_block_hash = proxy.get_block_hash(block_count)\n",
    "print(f\"Latest block hash: {latest_block_hash}\")\n",
    "\n",
    "# Get the block data\n",
    "block_data = proxy.get_block(latest_block_hash)\n",
    "\n",
    "# Display some block information\n",
    "print(f\"Block version: {block_data['version']}\")\n",
    "print(f\"Block time: {block_data['time']}\")\n",
    "print(f\"Number of transactions: {len(block_data['tx'])}\")\n",
    "print(f\"Merkle root: {block_data['merkleroot']}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Wallet Operations\n",
    "\n",
    "Let's perform some wallet operations."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Get wallet information\n",
    "wallet_info = proxy.get_wallet_info()\n",
    "print(f\"Wallet name: {wallet_info['walletname']}\")\n",
    "print(f\"Balance: {wallet_info['balance']} BTC\")\n",
    "\n",
    "# Generate a new address\n",
    "new_address = proxy.get_new_address(label=\"example\")\n",
    "print(f\"New address: {new_address}\")\n",
    "\n",
    "# List unspent transaction outputs (UTXOs)\n",
    "utxos = proxy.list_unspent()\n",
    "print(f\"Number of UTXOs: {len(utxos)}\")\n",
    "if len(utxos) > 0:\n",
    "    print(f\"First UTXO: {utxos[0]}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Working with Transactions\n",
    "\n",
    "Now let's create and send a transaction."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# This is a complete example of creating and sending a transaction\n",
    "# You should have at least one UTXO in your wallet to run this\n",
    "\n",
    "# First, check if we have any UTXOs\n",
    "utxos = proxy.list_unspent()\n",
    "if len(utxos) == 0:\n",
    "    print(\"No UTXOs available. Please send some funds to your wallet.\")\n",
    "else:\n",
    "    # Select the first UTXO\n",
    "    utxo = utxos[0]\n",
    "    \n",
    "    # Create a recipient address (here we use a new address from our own wallet for demonstration)\n",
    "    recipient_address = proxy.get_new_address(label=\"recipient\")\n",
    "    \n",
    "    # Prepare inputs and outputs\n",
    "    inputs = [{\"txid\": utxo[\"txid\"], \"vout\": utxo[\"vout\"]}]\n",
    "    \n",
    "    # Calculate amount to send (original amount minus fee)\n",
    "    fee = 0.0001  # Fixed fee for simplicity\n",
    "    amount_to_send = utxo[\"amount\"] - fee\n",
    "    \n",
    "    # Create outputs\n",
    "    outputs = {recipient_address: amount_to_send}\n",
    "    \n",
    "    # Create a raw transaction\n",
    "    raw_tx = proxy.create_raw_transaction(inputs, outputs)\n",
    "    print(f\"Raw transaction created:\\n{raw_tx}\")\n",
    "    \n",
    "    # Sign the transaction\n",
    "    signed_tx = proxy.sign_raw_transaction_with_wallet(raw_tx)\n",
    "    if signed_tx[\"complete\"]:\n",
    "        print(\"Transaction signed successfully!\")\n",
    "        \n",
    "        # Uncomment the following line to actually send the transaction\n",
    "        # tx_id = proxy.send_raw_transaction(signed_tx[\"hex\"])\n",
    "        # print(f\"Transaction sent! TXID: {tx_id}\")\n",
    "        \n",
    "        print(\"Transaction not sent (code is commented out for safety).\")\n",
    "    else:\n",
    "        print(\"Failed to sign transaction.\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Using Direct RPC Methods\n",
    "\n",
    "The `NodeProxy` class provides convenient methods for common operations, but you can also call any RPC method directly."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Call getnetworkinfo directly\n",
    "network_info = proxy.call('getnetworkinfo')\n",
    "print(f\"Bitcoin Core version: {network_info['version']}\")\n",
    "print(f\"Protocol version: {network_info['protocolversion']}\")\n",
    "print(f\"Connections: {network_info['connections']}\")\n",
    "\n",
    "# Alternative way to call RPC methods directly\n",
    "uptime = proxy('uptime')\n",
    "print(f\"Node uptime: {uptime} seconds\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Error Handling\n",
    "\n",
    "Let's see how to handle errors when working with the NodeProxy."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "from bitcoinutils.proxy import RPCError\n",
    "\n",
    "try:\n",
    "    # Try to get a non-existent block\n",
    "    block = proxy.get_block(\"0000000000000000000000000000000000000000000000000000000000000000\")\n",
    "except RPCError as e:\n",
    "    print(f\"RPC Error: {e}\")\n",
    "    if e.code is not None:\n",
    "        print(f\"Error code: {e.code}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Conclusion\n",
    "\n",
    "The `NodeProxy` class provides a convenient interface to interact with a Bitcoin node. It offers methods for common operations as well as the ability to call any RPC method directly.\n",
    "\n",
    "For more information, refer to the [Bitcoin Core RPC documentation](https://developer.bitcoin.org/reference/rpc/index.html)."
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.7.6"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}